<h1>모델학습</h1>

<h3>학습할 모델 자료 만들기</h3>

In [39]:
from ratsnlp import nlpbook

In [40]:
import pandas as pd
import torch

print(torch.backends.mps.is_built())
mps_device = torch.device("mps")

True


In [41]:
#기존에 있는 자료 사용
from tqdm import tqdm, trange
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("nsmc")
raw_datasets

Found cached dataset nsmc (/Users/sudong/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [42]:
checkpoint = "monologg/koelectra-base-generator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["document"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loading configuration file config.json from cache at /Users/sudong/.cache/huggingface/hub/models--monologg--koelectra-base-generator/snapshots/fe6a7147be11ae58af0f78206f558c8e31e8c5c9/config.json
Model config ElectraConfig {
  "_name_or_path": "monologg/koelectra-base-generator",
  "architectures": [
    "ElectraForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32200
}

loading fil

In [43]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [44]:
tokenized_datasets["train"][0]["document"]

'아 더빙.. 진짜 짜증나네요 목소리'

In [45]:
tokenized_datasets["train"][0]["input_ids"]

[2, 3360, 28709, 18, 18, 12704, 29334, 5853, 11852, 21747, 3]

<h3>데이터를 활용해서 모델 학습시키기</h3>

1. trainig arguments 설정

In [46]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    # "test-trainer",
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training 
    learning_rate=3e-5,
    weight_decay=0.01,               #하이퍼 파라미터 # strength of weight decay 
    per_device_train_batch_size=128,   # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    warmup_steps=1000,               # number of warmup steps for learning rate scheduler
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after each epoch
    gradient_accumulation_steps=64,  # total number of steps before back propagation
    # fp16=True,                       # Use mixed precision
    # fp16_opt_level="02",             # mixed precision mode
    run_name="ProBert-BFD-MS",       # experiment name
    seed=3,                           # Seed for experiment reproducibility 3x3
    # push_to_hub=True,
    # gradient_checkpointing=True # 메모리 절약 대신 느려짐
    # lr_scheduler_type='cosine',
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [47]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(mps_device)

loading configuration file config.json from cache at /Users/sudong/.cache/huggingface/hub/models--monologg--koelectra-base-generator/snapshots/fe6a7147be11ae58af0f78206f558c8e31e8c5c9/config.json
Model config ElectraConfig {
  "_name_or_path": "monologg/koelectra-base-generator",
  "architectures": [
    "ElectraForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32200
}

loading wei

In [48]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [49]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: id, document. If id, document are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 150000
  Num Epochs = 1
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 8192
  Gradient Accumulation steps = 64
  Total optimization steps = 18
  Number of trainable parameters = 34866178
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


  0%|          | 0/18 [00:00<?, ?it/s]

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: id, document. If id, document are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 128


  0%|          | 0/391 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.694394052028656, 'eval_runtime': 321.932, 'eval_samples_per_second': 155.312, 'eval_steps_per_second': 1.215, 'epoch': 0.98}
{'train_runtime': 3810.2034, 'train_samples_per_second': 39.368, 'train_steps_per_second': 0.005, 'train_loss': 0.6944135030110677, 'epoch': 0.98}


TrainOutput(global_step=18, training_loss=0.6944135030110677, metrics={'train_runtime': 3810.2034, 'train_samples_per_second': 39.368, 'train_steps_per_second': 0.005, 'train_loss': 0.6944135030110677, 'epoch': 0.98})

In [50]:
#학습된 모델의 파라미터 results파일에 저장하기
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained('./results')

Configuration saved in ./results/config.json
Model weights saved in ./results/pytorch_model.bin


<h3>모델 가져와서 쓰기</h3>


In [51]:
# loading the model you previously trained
model = AutoModelForSequenceClassification.from_pretrained('./results')

loading configuration file ./results/config.json
Model config ElectraConfig {
  "_name_or_path": "./results",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32200
}

loading weights file ./results/pytorch_model.bin
All model checkpoint weights were used whe

In [52]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
MODEL_NAME = "monologg/koelectra-base-generator"

tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-generator")

# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
from transformers import pipeline,TextClassificationPipeline

classifier = TextClassificationPipeline(model = model, tokenizer = tokenizer)

loading configuration file config.json from cache at /Users/sudong/.cache/huggingface/hub/models--monologg--koelectra-base-generator/snapshots/fe6a7147be11ae58af0f78206f558c8e31e8c5c9/config.json
Model config ElectraConfig {
  "_name_or_path": "monologg/koelectra-base-generator",
  "architectures": [
    "ElectraForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32200
}

loading fil

In [53]:
classifier

In [54]:
print(classifier("i love him"))
print(classifier("i hate him"))

[{'label': 'LABEL_0', 'score': 0.5203970074653625}]
[{'label': 'LABEL_0', 'score': 0.5212336778640747}]


In [55]:
print(tokenized_datasets["test"])

Dataset({
    features: ['id', 'document', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50000
})


In [57]:
print(classifier("이쁘고 좋아요~~~씻기도 편하고 아이고 이쁘다고 자기방에 갖다놓고 잘써요~^^"))
print(classifier("와 너 진짜 별로다"))
print(classifier("그냥저냥 보통인데?"))
print(classifier("이보다 재미있을 수 없다"))
print(classifier("재미있긴 한데 약간 아쉽다"))
print(classifier("아 더빙 ...진짜 짜증나네 목소리"))

[{'label': 'LABEL_0', 'score': 0.5266706347465515}]
[{'label': 'LABEL_0', 'score': 0.5189048051834106}]
[{'label': 'LABEL_0', 'score': 0.5127888917922974}]
[{'label': 'LABEL_0', 'score': 0.5154189467430115}]
[{'label': 'LABEL_0', 'score': 0.5136963129043579}]
[{'label': 'LABEL_0', 'score': 0.5161553025245667}]
